In [1]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [2]:
os.path.join('data','train.csv')

'data\\train.csv'

In [5]:
df = pd.read_csv('D:\\Coding\\projects\\DeepLearning\\toxic\\data\\train.csv\\train.csv')

In [6]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


Tokenizing

In [7]:
from tensorflow.keras.layers import TextVectorization

In [8]:
X = df['comment_text']
y = df[df.columns[2:]].values

In [9]:
MAX_Features = 200000 # number of words in the vocab

In [10]:
vectorizer = TextVectorization(max_tokens=MAX_Features,
                               output_sequence_length=1800,
                               output_mode='int')

In [13]:
type(X)
type(X.values)

numpy.ndarray

In [11]:
vectorizer.adapt(X.values)

In [16]:
vectorizer("I love you my friend")[:5]

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([  8, 454,   7,  28, 895], dtype=int64)>

In [14]:
vectorized_text = vectorizer(X.values)

In [23]:
#creating the dataset
#MCSHBAP - map, cache, shuffle, batch, prefetch  from_tensor_slices, list_file
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) # helps bottlenecks

In [25]:
batch_x,batch_y=dataset.as_numpy_iterator().next()

In [28]:
batch_x.shape #vectorized text
batch_y.shape #labels

(16, 6)

In [29]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [31]:
train_generator=train.as_numpy_iterator()
train_generator.next()#loops trough batches

(array([[  1204,      5,      8, ...,      0,      0,      0],
        [     8,   1107,      3, ...,      0,      0,      0],
        [  5299,    386,  55793, ...,      0,      0,      0],
        ...,
        [    90,      7,     20, ...,      0,      0,      0],
        [   437,     34,     14, ...,      0,      0,      0],
        [   451, 126474, 181491, ...,      0,      0,      0]], dtype=int64),
 array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]], dtype=int64))

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [32]:
model = Sequential()
# Create the embedding layer 
model.add(Embedding(MAX_Features+1, 32))

# Bidirectional LSTM Layer
#biderection layer enables passing info in both direction (since words have meaning according to their sequence)
model.add(Bidirectional(LSTM(32, activation='tanh')))

# Feature extractor Fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
# Final layer maps to the number of outputs
model.add(Dense(6, activation='sigmoid'))

In [33]:
y[0]

array([0, 0, 0, 0, 0, 0], dtype=int64)

In [34]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')

In [35]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          6400032   
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               16640     
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 128)               8320      
                                                                 
 dense_5 (Dense)             (None, 256)               33024     
                                                                 
 dense_6 (Dense)             (None, 128)               32896     
                                                                 
 dense_7 (Dense)             (None, 6)                 774       
                                                      

In [36]:
history = model.fit(train, epochs=1, validation_data=val)

6981/6981 [==============================] - 57692s 8s/step - loss: 0.0622 - val_loss: 0.0463


In [37]:
history.history

{'loss': [0.0622388981282711], 'val_loss': [0.046250469982624054]}

In [24]:
import pickle
with open('happy_sad.pkl','wb') as f:
    pickle.dump(history,f)

INFO:tensorflow:Assets written to: ram://794be465-f2f8-4ee1-a3d5-3f53a427c676/assets


INFO:tensorflow:Assets written to: ram://794be465-f2f8-4ee1-a3d5-3f53a427c676/assets


In [38]:
from matplotlib import pyplot as plt

making predictions

In [41]:
batch=test.as_numpy_iterator().next()

In [40]:
input_text=vectorizer("You suck")

In [42]:
input_text

<tf.Tensor: shape=(1800,), dtype=int64, numpy=array([  7, 396,   0, ...,   0,   0,   0], dtype=int64)>

In [43]:
model.predict(np.array([input_text]))

1/1 [==============================] - 7s 7s/step


array([[0.98758143, 0.06713213, 0.85757554, 0.01025962, 0.6928107 ,
        0.08892335]], dtype=float32)

In [44]:
#model.predict(input_text) wont run as model expects input in the form of batch
model.predict(np.expand_dims(input_text,0))

1/1 [==============================] - 0s 247ms/step


array([[0.98758143, 0.06713213, 0.85757554, 0.01025962, 0.6928107 ,
        0.08892335]], dtype=float32)

In [26]:
df.columns[2:]

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [27]:
batch_X,batch_y=test.as_numpy_iterator().next()

In [ ]:
model.predict(batch_X)

In [30]:
batch_y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 1],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

In [29]:
(model.predict(batch_X)>0.5).astype(int)

1/1 [==============================] - 1s 529ms/step


array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [31]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [32]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [35]:
for batch in test.as_numpy_iterator(): 
    # Unpack the batch 
    X_true, y_true = batch
    # Make a prediction 
    yhat = model.predict(X_true)
    
    # Flatten the predictions
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    
    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

1/1 [==============================] - 0s 324ms/step


In [36]:
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

Precision: 0.8500691652297974, Recall:0.6786660552024841, Accuracy:0.48546740412712097


In [45]:
model.save('toxicity2.h5')

In [47]:
model = tf.keras.models.load_model('toxicity2.h5')